In [21]:
import sys
import os
# Turn off warnings and errors due to TF libraries
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
import time
import datetime
import csv
from random import shuffle
import tensorflow as tf
# import internal scripts
from tools.tools import *
from test import test

In [22]:
config = {
"train_dir"   : 'data/train',
"valid_dir"   : 'data/valid',
"dataset" : 'mu200_1pT',
"log_dir" : 'logs/test_CGNN/',
"run_type" : 'new_run',
"gpu" : '-1',
"n_files" : 100,
"n_valid" : 50,
"n_train" : 50,
"lr_c" : 0.01,
"batch_size" : 1,
"n_iters" : 3,
"n_epoch" : 5,
"TEST_every" : 50,
"hid_dim" : 4,
"network" : 'CGNN',
"optimizer"  : 'Adam',
"loss_func"  : 'BinaryCrossentropy',
"n_thread"  : 4,
"log_verbosity": 2,
}
tools.config = config

In [23]:
os.environ["CUDA_VISIBLE_DEVICES"] = config['gpu']
USE_GPU = (config['gpu']  != '-1')

In [24]:
    os.environ['OMP_NUM_THREADS'] = str(config['n_thread'])  # set num workers
    tf.config.threading.set_intra_op_parallelism_threads(config['n_thread'])
    tf.config.threading.set_inter_op_parallelism_threads(config['n_thread'])

In [25]:
    # Load the network
    if config['network'] == 'QGNN':
        from qnetworks.QGNN import GNN
        GNN.config = config
    elif config['network'] == 'CGNN':
        from qnetworks.CGNN import GNN
        GNN.config = config
    else: 
        print('Wrong network specification!')
        sys.exit()

In [26]:
    # setup model
    model = GNN()

    # load data
    train_data = get_dataset(config['train_dir'], config['n_train'])
    train_list = [i for i in range(config['n_train'])]

In [27]:
    # execute the model on an example data to test things
    X, Ri, Ro, y = train_data[0]
    model([map2angle(X), Ri, Ro])


<tf.Tensor: shape=(8353, 1), dtype=float32, numpy=
array([[0.55199516],
       [0.551275  ],
       [0.5588722 ],
       ...,
       [0.47679737],
       [0.5069042 ],
       [0.4788517 ]], dtype=float32)>

In [28]:
    print(model.summary())

Model: "GNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputNet (Sequential)        (None, 4)                 16        
_________________________________________________________________
EdgeNet (EdgeNet)            multiple                  65        
_________________________________________________________________
NodeNet (NodeNet)            multiple                  108       
Total params: 189
Trainable params: 189
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
def check_type(name, x):
    print(x)
    print("name:", name)
    print("type:", type(x))
    print(len(x))
    print(np.array(x).shape)
    print(np.array(x).dtype)
    #if (np.ndim(np.array(x)) == 2):
    if isinstance(x, np.ndarray): 
        print("MIN", np.array(x).min())
        print("MAX", np.array(x).max())
    #else:
    #    print("ndim:", np.ndim(np.array(x)))


In [35]:
X, Ri, Ro, y = train_data[1]
check_type("X", X)

[[ 0.03201562  0.2093877  -0.205722  ]
 [ 0.072587    0.20926541 -0.48162702]
 [ 0.07247818  0.6999745  -0.104102  ]
 ...
 [ 0.5030189   0.01880868  0.61259997]
 [ 0.6623555   0.03205843  0.8066    ]
 [ 0.82296497  0.0452814   1.003     ]]
name: X
<class 'numpy.ndarray'>
5923
(5923, 3)
float32
MIN -1.081
MAX 1.0785999


In [39]:
check_type("Ri", Ri)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
name: Ri
type: <class 'numpy.ndarray'>
5923
(5923, 9269)
float32
MIN 0.0
MAX 1.0


In [40]:
check_type("Ro", Ro)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
name: Ro
type: <class 'numpy.ndarray'>
5923
(5923, 9269)
float32
MIN 0.0
MAX 1.0


In [42]:
check_type("y", y)

[1. 1. 0. ... 1. 1. 1.]
name: y
type: <class 'numpy.ndarray'>
9269
(9269,)
float32
MIN 0.0
MAX 1.0


In [45]:
X_angle = map2angle(X)
check_type("X_angle", X_angle)

[[0.02910511 0.60469383 0.40649003]
 [0.06598818 0.60463274 0.28107864]
 [0.06588925 0.84998727 0.45268092]
 ...
 [0.4572899  0.50940436 0.77845454]
 [0.6021413  0.51602924 0.86663634]
 [0.74814993 0.5226407  0.95590913]]
name: X_angle
type: <class 'numpy.ndarray'>
5923
(5923, 3)
float32
MIN 5.620718e-05
MAX 0.9999594
